In [71]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio
#needs to be installed while building docker or while running notwbook
# import librosa
from pydub import AudioSegment
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [72]:
data_dir  = os.path.join('..','data') 

In [73]:
df = pd.read_excel(os.path.join(data_dir,'VSD.xlsx'),sheet_name='read_dataset')

In [74]:
df.head()

,File_segment_name,Duration,Violence_start,Violence_end,Violence_duration
0,angry_011,117.0,21.0,38.0,17.0
1,angry_011,117.0,40.0,55.0,15.0
2,angry_011,117.0,60.0,79.0,19.0
3,angry_011,117.0,85.0,95.0,10.0
4,angry_011,117.0,101.0,110.0,9.0


In [75]:
df.duplicated().sum()

0

In [76]:
df.isnull().sum()

File_segment_name    0
Duration             0
Violence_start       0
Violence_end         0
Violence_duration    0
dtype: int64

In [77]:
def get_dir(filepath):
    toxic_dir = os.path.join(filepath,'toxic')
    non_toxic_dir= os.path.join(filepath,'non_toxic')
    if not os.path.exists(toxic_dir):
        os.mkdir(toxic_dir)
    if not os.path.exists(non_toxic_dir):
        os.mkdir(non_toxic_dir)
    return toxic_dir,non_toxic_dir

def segment_toxic_audio(filepath,df,toxic_dir):
    for index,row in df.iterrows():
        t1 = 1000 * row[2]
        t2 = 1000 * row[3]
        audio  = AudioSegment.from_wav(os.path.join(filepath,str(row[0])+'.wav'))
        toxic = audio[t1:t2]
        a= 0 
        while len(toxic)>4000:
            toxic[:4000].export(os.path.join(toxic_dir,'toxic_'+str(index)+'_'+str(a)+".wav"),format = 'wav')
            toxic = toxic[4000:]
            a+=1
def segment_non_toxic_audio(filepath,df,non_toxic_dir):
    no = 0 #for naming purposes
    unique_aud = df['File_segment_name'].unique()
    for aud in unique_aud:
        t1 = 0
        for row in df[df['File_segment_name']== aud ].values:
            no+=1
            t2 = 1000 * row[2]
            audio  = AudioSegment.from_wav(os.path.join(filepath,str(row[0])+'.wav'))
            non_toxic = audio[t1:t2] if t2!=0 else audio
            a= 0 
            while len(non_toxic)>4000:
                non_toxic[:4000].export(os.path.join(non_toxic_dir,'non_toxic'+'_'+str(no)+'_'+str(a)+".wav"),format = 'wav')
                non_toxic = non_toxic[4000:]
                a+=1
            t1 = 1000* row[3]

In [78]:
def data_segmentation(data_path,df):
    t_path,nt_path = get_dir(data_path)
    audio_path = os.path.join(data_path,'audios_VSD','audios_VSD')
    segment_toxic_audio(audio_path,df,t_path)
    segment_non_toxic_audio(audio_path,df,nt_path)

In [79]:
data_segmentation(data_dir,df)

In [15]:
#Using tensorflow instead of librosa
